<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [16]:
%%sql
SELECT p_id, date, time, title, year, t_name, (capacity - count(t_id)) AS remaining_seats
FROM   performances
JOIN   movies
USING  (imdb_nbr)
JOIN   theaters
USING  (t_name)
LEFT JOIN tickets
USING  (performance_nbr)
GROUP BY performance_nbr

 * sqlite:///movies.sqlite
Done.


performance_nbr,start_date,start_time,imdb_nbr,t_name,reminingSeats
None,None,None,None,None,None


+ Show the performance dates for one of the movies.

In [7]:
%%sql

select *
from tickets


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: performances.performance_nbr
[SQL: INSERT
INTO performances
VALUES('fd8d3790e16c110fc8ac50d84f408a3f','12:23','tt5580390' ,'Kino' ,'2019-02-23');]
(Background on this error at: http://sqlalche.me/e/gkpj)

+ Show all data concerning performances at a given theatere
  on a given date.

In [ ]:
%%sql
SELECT theaters.*, performances.*
FROM theaters, performances
WHERE theaters.t_name = performances.t_name
AND theaters.t_name = 'idet'
AND performances.start_date = '2020-02-02'

+ List all customers

In [ ]:
%%sql
SELECT *
FROM customers

+ List all tickets

In [ ]:
%%sql
SELECT *
FROM tickets

+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [ ]:
%%sql
INSERT
INTO tickets (performance_nbr, username)
VALUES (3, 'hackerman1337')


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2020), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [27]:
%%sql
SELECT *
FROM tickets


 * sqlite:///movies.sqlite
Done.


id,performance_nbr,username
cc5366de63fc0f029af0ee37a6bb96e6,f64ef24423752b527b1257e7413b718,alice
9a0e8fba6a1847dbd54155172469f37e,f64ef24423752b527b1257e7413b718,alice
bfe249fa0a46d6e90ddc323fc78cf410,f64ef24423752b527b1257e7413b718,alice
200c7c9a968f80143504e16a4169d542,f64ef24423752b527b1257e7413b718,alice
9297c88ce500adfe44427fe999c2205f,f64ef24423752b527b1257e7413b718,alice


+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql
INSERT
INTO theaters
VALUES ('e:a', 120), ('e:a',140);

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [24]:
%%sql
select *
from performances



 * sqlite:///movies.sqlite
Done.


performance_nbr,start_time,imdb_nbr,t_name,start_date
f64ef24423752b5275b1257e7413b718,19:00,tt5580390,Kino,2019-02-22
a944bc94a55e1d6291e30fdc4f4e2db6,12:23,tt5580390,Kino,2019-02-23
fd8d3790e16c110fc8ac50d84f408a3f,12:23,tt5580390,Kino,2019-02-23


+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [26]:
%%sql
INSERT
INTO tickets (performance_nbr, username)
VALUES(fd8d3790e16c110fc8ac50d84f408a3f, 'Bob');

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) no such column: fd8d3790e16c110fc8ac50d84f408a3f
[SQL: INSERT
INTO tickets (performance_nbr, username)
VALUES(fd8d3790e16c110fc8ac50d84f408a3f, 'Bob');]
(Background on this error at: http://sqlalche.me/e/e3q8)
